# 神经网络模型：

In [189]:
import torch

class Para:
    # tensor_board_log_dir = 'runs/exp0'
    feature_column_start_name = 'ep_ratio_ttm'
    feature_column_end_name = 'BR'

    # 模型设置
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    classification = 2 # 2, 3

    # 权重
    cross_weight = list()
    if classification == 3:
        cross_weight = [1.0, 1.0 ,1.0]
    elif classification == 2:
        cross_weight = [1.0, 1.0]
    elif classification == 5:
        cross_weight = [1.0, 1.0, 1.0, 1.0, 1.0]

    batch_size = 16
    lr = 1e-3
    drop = 0.5
    epochs = 10

    # 数据集设置
    month_in_sample = range(0, 1)
    # month_test = range(36, 48)

    percent_cv = 0.1 # 10% cross validation

    data_path = 'data/sq_space_1d_rate_20d_17-21_pre'


    seed = 2022
    torch.manual_seed(seed)

    info_str0 = data_path[5:7]+'_'+'c'+str(classification)+'_s'+str(percent_cv)
    info_str1 = '_b'+str(batch_size)+'_lr'+str(lr)+'_d'+str(drop)+'_e'+str(epochs)
    info_str = info_str0 + info_str1

    save_model_path = 'models/'+'model_'+info_str+'.pth'

para = Para()
print(para.info_str)
print(para.save_model_path)
print(torch.cuda.is_available())
print(torch.cuda.device_count())
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))
    print(torch.cuda.current_device())

sq_c2_s0.1_b16_lr0.001_d0.5_e10
models/model_sq_c2_s0.1_b16_lr0.001_d0.5_e10.pth
True
1
GeForce MX150
0


### 构建训练集

In [190]:
import numpy as np
import pandas as pd

data_in_sample = None
for i_month in para.month_in_sample:
    file_name = para.data_path + '/' + str(i_month) + '.csv'
    data_curr_month = pd.read_csv(file_name)

    data_curr_month = data_curr_month.dropna(axis=0)

    data_curr_month.insert(loc=0, column='return_bin', value=np.nan)

    data_curr_month.loc[data_curr_month['yield_rate']>0, 'return_bin'] = 0
    data_curr_month.loc[data_curr_month['yield_rate']<=0, 'return_bin'] = 1

    if i_month == para.month_in_sample[0]:
        data_in_sample = data_curr_month
    else:
        data_in_sample = pd.concat([data_in_sample, data_curr_month])
        # data_in_sample = data_in_sample.append(data_curr_month)

data_in_sample

,return_bin,order_book_id,board_type,sector_code,month,date,yield_rate,ep_ratio_ttm,pb_ratio_ttm,sp_ratio_ttm,...,RSI10,SY,BIAS20,VOL30,VOL60,VOL120,VOLT20,VOLT60,AR,BR
0,0.0,600104.XSHG,MainBoard,ConsumerDiscretionary,0,2017-01-03,0.809567,1.284306,0.358685,-0.160216,...,0.988857,0.500157,0.253561,1.023958,0.067001,-0.342131,-0.402515,-0.940741,-0.185952,-0.682637
1,0.0,600104.XSHG,MainBoard,ConsumerDiscretionary,1,2017-01-04,0.428012,1.152139,0.425961,-0.255248,...,1.218035,0.500157,0.657302,1.056231,0.093217,-0.333081,-0.428033,-0.957848,0.489018,-0.155550
2,0.0,600104.XSHG,MainBoard,ConsumerDiscretionary,2,2017-01-05,0.509194,1.230912,0.385595,-0.198608,...,0.870098,-0.250079,0.441853,0.991284,0.105760,-0.330882,-0.477653,-0.993590,-0.566329,-0.978739
3,0.0,600104.XSHG,MainBoard,ConsumerDiscretionary,3,2017-01-06,0.638719,1.277592,0.362049,-0.165043,...,0.407437,-0.250079,0.348336,0.794401,0.121854,-0.347766,-0.610628,-1.023715,-0.485009,-0.883754
4,0.0,600104.XSHG,MainBoard,ConsumerDiscretionary,4,2017-01-09,0.316583,1.194613,0.404096,-0.224708,...,0.954654,0.500157,0.626933,0.716922,0.148316,-0.346567,-0.731410,-1.051159,-0.492318,-1.017275
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1189,0.0,600104.XSHG,MainBoard,ConsumerDiscretionary,1192,2021-11-29,0.417490,0.131692,-1.265140,1.100538,...,-1.323734,-0.250079,-1.043171,1.331187,1.138590,1.174348,0.272634,-0.653109,-0.360370,-0.445948
1190,0.0,600104.XSHG,MainBoard,ConsumerDiscretionary,1193,2021-11-30,0.532614,0.179925,-1.281825,1.149883,...,-1.036148,-1.000314,-1.183964,1.311968,1.097471,1.174860,0.317672,-0.699096,-0.414854,-0.318488
1191,0.0,600104.XSHG,MainBoard,ConsumerDiscretionary,1194,2021-12-01,0.207170,0.097650,-1.253223,1.065712,...,-0.840902,-0.250079,-0.918093,1.276317,1.038991,1.174089,0.317672,-0.722895,-0.199559,-0.229953
1192,1.0,600104.XSHG,MainBoard,ConsumerDiscretionary,1195,2021-12-02,-0.091058,-0.035215,-1.205553,0.929787,...,-0.097430,-0.250079,-0.471171,1.291874,1.040739,1.191806,0.322346,-0.752404,-0.000075,0.027994


In [191]:
X_in_sample = data_in_sample.loc[:, para.feature_column_start_name: para.feature_column_end_name]
y_in_sample = data_in_sample.loc[:, 'return_bin']

from sklearn.model_selection import train_test_split

X_train, X_cv, y_train, y_cv = train_test_split(X_in_sample, y_in_sample, test_size=para.percent_cv, shuffle=False) # True, random_state=para.seed)
# X_train, X_cv, y_train, y_cv = train_test_split(X_in_sample, y_in_sample, test_size=para.percent_cv, shuffle=False)

X_train

,ep_ratio_ttm,pb_ratio_ttm,sp_ratio_ttm,MACD_DIFF,MACD_DEA,MACD_HIST,RSI10,SY,BIAS20,VOL30,VOL60,VOL120,VOLT20,VOLT60,AR,BR
0,1.284306,0.358685,-0.160216,-0.431525,-0.477758,0.030239,0.988857,0.500157,0.253561,1.023958,0.067001,-0.342131,-0.402515,-0.940741,-0.185952,-0.682637
1,1.152139,0.425961,-0.255248,-0.249418,-0.436122,0.451899,1.218035,0.500157,0.657302,1.056231,0.093217,-0.333081,-0.428033,-0.957848,0.489018,-0.155550
2,1.230912,0.385595,-0.198608,-0.145385,-0.380331,0.605792,0.870098,-0.250079,0.441853,0.991284,0.105760,-0.330882,-0.477653,-0.993590,-0.566329,-0.978739
3,1.277592,0.362049,-0.165043,-0.086603,-0.322995,0.622588,0.407437,-0.250079,0.348336,0.794401,0.121854,-0.347766,-0.610628,-1.023715,-0.485009,-0.883754
4,1.194613,0.404096,-0.224708,0.002761,-0.257813,0.707880,0.954654,0.500157,0.626933,0.716922,0.148316,-0.346567,-0.731410,-1.051159,-0.492318,-1.017275
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1069,0.594835,-1.120446,1.256091,0.037133,0.018016,0.059125,-0.558520,0.500157,0.079456,-0.134044,-0.373492,1.230104,-1.477572,-1.585713,0.709617,0.168198
1070,0.656485,-1.141604,1.316486,0.017431,0.018166,0.000856,-0.461329,0.500157,-0.112962,-0.236794,-0.378449,1.223895,-1.475688,-1.608194,0.734207,0.217079
1071,0.693237,-1.154049,1.352490,-0.015745,0.011115,-0.077414,-0.173590,0.500157,-0.238283,-0.292676,-0.390751,1.209950,-1.518329,-1.612158,0.842647,0.257739
1072,0.719180,-1.162761,1.377906,-0.053966,-0.002785,-0.151887,0.172847,-0.250079,-0.305984,-0.326746,-0.417233,1.176037,-1.492499,-1.616117,0.894925,0.327854


In [192]:
y_train

0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
1069    0.0
1070    0.0
1071    0.0
1072    0.0
1073    0.0
Name: return_bin, Length: 1074, dtype: float64

In [193]:
data_train = pd.concat([X_train, y_train], axis=1)
data_cv = pd.concat([X_cv, y_cv], axis=1)
data_train

,ep_ratio_ttm,pb_ratio_ttm,sp_ratio_ttm,MACD_DIFF,MACD_DEA,MACD_HIST,RSI10,SY,BIAS20,VOL30,VOL60,VOL120,VOLT20,VOLT60,AR,BR,return_bin
0,1.284306,0.358685,-0.160216,-0.431525,-0.477758,0.030239,0.988857,0.500157,0.253561,1.023958,0.067001,-0.342131,-0.402515,-0.940741,-0.185952,-0.682637,0.0
1,1.152139,0.425961,-0.255248,-0.249418,-0.436122,0.451899,1.218035,0.500157,0.657302,1.056231,0.093217,-0.333081,-0.428033,-0.957848,0.489018,-0.155550,0.0
2,1.230912,0.385595,-0.198608,-0.145385,-0.380331,0.605792,0.870098,-0.250079,0.441853,0.991284,0.105760,-0.330882,-0.477653,-0.993590,-0.566329,-0.978739,0.0
3,1.277592,0.362049,-0.165043,-0.086603,-0.322995,0.622588,0.407437,-0.250079,0.348336,0.794401,0.121854,-0.347766,-0.610628,-1.023715,-0.485009,-0.883754,0.0
4,1.194613,0.404096,-0.224708,0.002761,-0.257813,0.707880,0.954654,0.500157,0.626933,0.716922,0.148316,-0.346567,-0.731410,-1.051159,-0.492318,-1.017275,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1069,0.594835,-1.120446,1.256091,0.037133,0.018016,0.059125,-0.558520,0.500157,0.079456,-0.134044,-0.373492,1.230104,-1.477572,-1.585713,0.709617,0.168198,0.0
1070,0.656485,-1.141604,1.316486,0.017431,0.018166,0.000856,-0.461329,0.500157,-0.112962,-0.236794,-0.378449,1.223895,-1.475688,-1.608194,0.734207,0.217079,0.0
1071,0.693237,-1.154049,1.352490,-0.015745,0.011115,-0.077414,-0.173590,0.500157,-0.238283,-0.292676,-0.390751,1.209950,-1.518329,-1.612158,0.842647,0.257739,0.0
1072,0.719180,-1.162761,1.377906,-0.053966,-0.002785,-0.151887,0.172847,-0.250079,-0.305984,-0.326746,-0.417233,1.176037,-1.492499,-1.616117,0.894925,0.327854,0.0


In [194]:
from torch.utils.data import TensorDataset
import torch

X_train_ndarray = X_train.values
y_train_ndarray = y_train.values

train_dataset = TensorDataset(torch.from_numpy(X_train_ndarray).type(torch.FloatTensor), torch.from_numpy(y_train_ndarray).type(torch.LongTensor))

# for X_train_temp, y_train_temp in train_dataset:
#     print(X_train_temp, y_train_temp)
#     print(X_train_temp.dtype, y_train_temp.dtype)

In [195]:
X_cv_ndarray = X_cv.values
y_cv_ndarray = y_cv.values

cv_dataset = TensorDataset(torch.from_numpy(X_cv_ndarray).type(torch.FloatTensor), torch.from_numpy(y_cv.values).type(torch.LongTensor))

# for X_cv_temp, y_cv_temp in cv_dataset:
#     print(X_cv_temp, y_cv_temp)
#     print(X_cv_temp.dtype, y_cv_temp.dtype)

In [196]:
from torch.utils.data import DataLoader


train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=para.batch_size,
    shuffle=True
)

cv_dataloader = DataLoader(
    dataset=cv_dataset,
    batch_size=para.batch_size,
    shuffle=True
)

### 构建测试集

In [197]:
# data_test = None
# for i_month in para.month_test:
#
#     file_name = para.data_path + '/' + str(i_month) + '.csv'
#     data_curr_month = pd.read_csv(file_name)
#
#     data_curr_month = data_curr_month.dropna(axis=0)
#
#     data_curr_month = label_data(data=data_curr_month, percent_select=para.percent_select)
#
#     if i_month == para.month_test[0]:
#         data_test = data_curr_month
#     else:
#         data_test = pd.concat([data_test, data_curr_month])
#         # data_test = data_test.append(data_curr_month)
#
# X_test = data_test.loc[:, para.feature_column_start_name: para.feature_column_end_name]
# y_test = data_test.loc[:, 'return_bin']

In [198]:
# from torch.utils.data import TensorDataset
# import torch
#
#
# X_test_ndarray = X_test.values
# y_test_ndarray = y_test.values
#
# test_dataset = TensorDataset(torch.from_numpy(X_test_ndarray).type(torch.FloatTensor), torch.from_numpy(y_test_ndarray).type(torch.LongTensor))

In [199]:
# from torch.utils.data import DataLoader
#
#
# test_dataloader = DataLoader(
#     dataset=test_dataset,
#     batch_size=para.batch_size,
#     shuffle=True,
# )

### 构建神经网络

In [200]:
from my_utils.model_class import MLP

model = MLP(in_nums=len(X_train.columns), out_nums=para.classification, drop_p=para.drop)
# to device
model = model.to(device=para.device)
print(model)

MLP(
  (linear_stack): Sequential(
    (0): Linear(in_features=16, out_features=16, bias=True)
    (1): Dropout(p=0.5, inplace=False)
    (2): ReLU()
    (3): Linear(in_features=16, out_features=8, bias=True)
    (4): Dropout(p=0.5, inplace=False)
    (5): ReLU()
    (6): Linear(in_features=8, out_features=2, bias=True)
  )
)


### 训练与测试

In [201]:
import torchmetrics


def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    train_loss, correct = 0, 0

    # initialize metric
    train_precision = torchmetrics.Precision(average='none', num_classes=para.classification).to(device=para.device)

    for batch, (X, y) in enumerate(dataloader):

        # to device
        X = X.to(device=para.device)
        y = y.to(device=para.device)

        # compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        train_loss += loss_fn(pred, y).item()
        correct += (pred.argmax(1) == y).type(torch.float).sum().item()

        # metric on current batch
        train_precision(pred.argmax(1), y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # if batch % 10 == 0:
        #     loss, current = loss.item(), batch * len(X)
        #     print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

    train_loss /= num_batches
    correct /= size
    print(f"Train Error: \n    Accuracy: {(100*correct):>0.1f}%, Avg loss: {train_loss:>8f} \n")

    # metric on all batches using custom accumulation
    total_precision = train_precision.compute()
    print("Precision of every train dataset class: ", total_precision)
    print()

    return correct, train_loss, total_precision


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # initialize metric
    test_precision = torchmetrics.Precision(average='none', num_classes=para.classification).to(device=para.device)

    with torch.no_grad():
        for X, y in dataloader:

            # to device
            X = X.to(device=para.device)
            y = y.to(device=para.device)

            # compute prediction and loss
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

            # metric on current batch
            test_precision(pred.argmax(1), y)


    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n    Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

    # metric on all batches using custom accumulation
    total_precision = test_precision.compute()
    print("Precision of every test dataset class: ", total_precision)
    print()

    return correct, test_loss, total_precision

In [202]:
def select_df_to_dataloader(df: pd.DataFrame, select: int) -> DataLoader:

    df = df[df['return_bin'] == select]

    df_dataset = TensorDataset(
        torch.from_numpy(df.loc[:, para.feature_column_start_name: para.feature_column_end_name].values).type(torch.FloatTensor),
        torch.from_numpy(df.loc[:, 'return_bin'].values).type(torch.LongTensor))

    df_dataloader = DataLoader(
        dataset=df_dataset,
        batch_size=para.batch_size,
        shuffle=True,
    )

    return df_dataloader

In [203]:
# temp2_dataloader = select_df_to_dataloader(df=data_cv, select=2)
temp1_dataloader = select_df_to_dataloader(df=data_cv, select=1)
temp0_dataloader = select_df_to_dataloader(df=data_cv, select=0)

In [204]:
from torch import nn
import time
from torch.utils.tensorboard import SummaryWriter

# 计时
time_start = time.time()

# writer = SummaryWriter(para.tensor_board_log_dir)
writer = SummaryWriter()

# 损失函数
loss_fn = nn.CrossEntropyLoss()
# to device
loss_fn = loss_fn.to(device=para.device)

# 优化器
optimizer = torch.optim.Adam(model.parameters(), lr=para.lr)

epochs = para.epochs
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")

    model.train()
    accuracy_train, loss_train, precision_train = train_loop(train_dataloader, model, loss_fn, optimizer)
    model.eval()
    accuracy_cv, loss_cv, precision_cv = test_loop(cv_dataloader, model, loss_fn)

    # accuracy2 = test_loop(temp2_dataloader, model, loss_fn)
    # print('#')
    # accuracy1 = test_loop(temp1_dataloader, model, loss_fn)
    # accuracy0 = test_loop(temp0_dataloader, model, loss_fn)

    # 写入 tensorboard
    if para.classification == 2:

        writer.add_scalars(main_tag=para.info_str+'_evaluation/cv',
                           tag_scalar_dict={
                               'accuracy': accuracy_cv,
                               'precision0': precision_cv[0],
                               'precision1': precision_cv[1]},
                           global_step=t)

        writer.add_scalars(main_tag=para.info_str+'_evaluation/train',
                           tag_scalar_dict={
                               'accuracy': accuracy_train,
                               'precision0': precision_train[0],
                               'precision1': precision_train[1]},
                           global_step=t)

    elif para.classification == 3:

        writer.add_scalars(main_tag=para.info_str+'_evaluation/cv',
                           tag_scalar_dict={
                               'accuracy': accuracy_cv,
                               'precision0': precision_cv[0],
                               'precision1': precision_cv[1],
                               'precision2': precision_cv[2]},
                           global_step=t)

        writer.add_scalars(main_tag=para.info_str+'_evaluation/train',
                           tag_scalar_dict={
                               'accuracy': accuracy_train,
                               'precision0': precision_train[0],
                               'precision1': precision_train[1],
                               'precision2': precision_train[2]},
                           global_step=t)

    elif para.classification == 5:

        writer.add_scalars(main_tag=para.info_str+'_evaluation/cv',
                           tag_scalar_dict={
                               'accuracy': accuracy_cv,
                               'precision0': precision_cv[0],
                               'precision1': precision_cv[1],
                               'precision2': precision_cv[2],
                               'precision3': precision_cv[3],
                               'precision4': precision_cv[4]},
                           global_step=t)

        writer.add_scalars(main_tag=para.info_str+'_evaluation/train',
                           tag_scalar_dict={
                               'accuracy': accuracy_train,
                               'precision0': precision_train[0],
                               'precision1': precision_train[1],
                               'precision2': precision_train[2],
                               'precision3': precision_train[3],
                               'precision4': precision_train[4]},
                           global_step=t)

    writer.add_scalars(main_tag=para.info_str+'_loss/cv',
                       tag_scalar_dict={
                           'loss': loss_cv},
                       global_step=t)

    writer.add_scalars(main_tag=para.info_str+'_loss/train',
                       tag_scalar_dict={
                           'loss': loss_train},
                       global_step=t)
    writer.flush()

    time_end = time.time()
    print('Time cost = %fs' % (time_end - time_start))
    print()

writer.close()

print("Done!")

Epoch 1
-------------------------------
Train Error: 
    Accuracy: 50.9%, Avg loss: 0.695894 

Precision of every train dataset class:  tensor([0.5012, 0.5145], device='cuda:0')

Test Error: 
    Accuracy: 47.5%, Avg loss: 0.689949 

Precision of every test dataset class:  tensor([0.5116, 0.3824], device='cuda:0')

Time cost = 0.922371s

Epoch 2
-------------------------------
Train Error: 
    Accuracy: 52.3%, Avg loss: 0.689486 

Precision of every train dataset class:  tensor([0.5207, 0.5247], device='cuda:0')

Test Error: 
    Accuracy: 52.5%, Avg loss: 0.690198 

Precision of every test dataset class:  tensor([0.5444, 0.4667], device='cuda:0')

Time cost = 1.560776s

Epoch 3
-------------------------------
Train Error: 
    Accuracy: 56.5%, Avg loss: 0.680858 

Precision of every train dataset class:  tensor([0.5981, 0.5518], device='cuda:0')

Test Error: 
    Accuracy: 52.5%, Avg loss: 0.689209 

Precision of every test dataset class:  tensor([0.5408, 0.4545], device='cuda:0')



## 保存模型

In [205]:
torch.save(model.state_dict(), para.save_model_path)

print('Finish save model!')

Finish save model!


## captum

In [206]:
# # captum
# from captum.attr import IntegratedGradients
#
# ig = IntegratedGradients(model)

In [207]:
# temp = cv_dataloader.dataset.tensors[0]
# temp.requires_grad_()
# attr, delta = ig.attribute(temp,target=1, return_convergence_delta=True)
# attr = attr.detach().numpy()

In [208]:
# # Helper method to print importances and visualize distribution
# def visualize_importances(feature_names, importances, title="Average Feature Importances", plot=True, axis_title="Features"):
#     print(title)
#     for i in range(len(feature_names)):
#         print(feature_names[i], ": ", '%.3f'%(importances[i]))
#     y_pos = (np.arange(len(feature_names)))
#     if plot:
#         plt.figure(figsize=(20,6))
#         plt.barh(y_pos, importances, align='center')
#         plt.yticks(y_pos, feature_names)
#         plt.ylabel(axis_title)
#         plt.grid(axis='y')
#         plt.title(title)
# visualize_importances(feature_names=X_cv.columns.values.tolist(), importances=np.mean(attr, axis=0))

In [209]:
# X_cv.columns.values.tolist()

In [210]:
# loss = nn.CrossEntropyLoss()
# input = torch.Tensor(
#     [[-0.0441,  0.0773],
#     [-0.0781, -0.1772],
#     [-0.1319, -0.0432],
#     [-0.0714, -0.1261],
#     [-0.0806, -0.1370],
#     [-0.1730, -0.1472],
#     [-0.0350, -0.0507],
#     [-0.1149, -0.2248]])
# # input = input.reshape(-1,4)
# target = torch.Tensor([0, 1, 1, 0, 0, 0, 0, 0]).type(torch.LongTensor)
# print(input.dtype)
# print(target.dtype)
# output = loss(input, target)
# print(input, target, output)

In [211]:
# # Example of target with class indices
# loss = nn.CrossEntropyLoss()
# input = torch.randn(3, 5, requires_grad=True)
# target = torch.Tensor([1,4,1]).type(torch.LongTensor)
# output = loss(input, target)
# print(input,target,output)

In [212]:
# loss = nn.BCEWithLogitsLoss()
# input = torch.Tensor([0.5, 0.4, 0.3])
# target = torch.Tensor([0])
# output = loss(input, target)
# print(input, target, output)